In [ ]:
from read import read_datasets

import torch
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
row = 28  #图片像素行数
cols = 28  #图片像素列数
image_sum = 600  #已知训练集有60000个图，任取一段
test_sum = 100  #已知测试集有10000个图，任取一段

input_dim = 784  #输入神经元数
#input_dim = row*cols
output_dim = 10  #输出神经元数
hidden_dim = 150  #隐藏神经元数

In [ ]:
#读取训练集
datasets_train = read_datasets('data/image_idx3/train-images.idx3-ubyte',
                               'data/lable_idx1/train-labels.idx1-ubyte')
#datasets_train

In [ ]:
#读取测试集
datasets_test = read_datasets('data/image_idx3/t10k-images.idx3-ubyte',
                              'data/lable_idx1/t10k-labels.idx1-ubyte')
#datasets_test

In [18]:
#定义模型
model = torch.nn.Sequential(torch.nn.Linear(input_dim, hidden_dim),
                            torch.nn.Sigmoid(),
                            torch.nn.Linear(hidden_dim, output_dim))
loss_func = torch.nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#训练网络
error_rate = 0
out = 1
random_index = 0
while (out):
    for epoch in range(image_sum):
        x_input = torch.tensor(datasets_train[0][epoch+random_index],
                               dtype=torch.float,
                               requires_grad=True)
        y_true = torch.tensor(datasets_train[1][epoch+random_index],
                              dtype=torch.float,
                              requires_grad=True)
        y_pre = model(x_input)
        #计算误差
        loss = loss_func(y_pre, y_true)
        #计算梯度
        optimizer.zero_grad()
        #更新参数
        loss.backward(retain_graph=True)
        optimizer.step()
    #验证错误率，当不在下降就退出循环
    pre_true = 0
    pre_wrong = 0
    for epoch in range(test_sum):
        x_input = torch.tensor(datasets_test[0][epoch],
                               dtype=torch.float,
                               requires_grad=False)
        y_true = torch.tensor(datasets_test[1][epoch],
                              dtype=torch.int,
                              requires_grad=False)
        y_pre = torch.argmax(model(x_input))
        if y_true == y_pre:
            pre_true += 1
        else:
            pre_wrong += 1
    new_rate = pre_wrong / (pre_true + pre_wrong)
    if abs(new_rate - error_rate) < 0.01:
        out = 0
    else:
        #不满足退出条件的的话就在训练集里面再随机截取一段继续训练
        random_index = np.random.randint(60000 - image_sum)
print('error rate: %.3f'% error_rate)


In [ ]:
epoch = 5
x_input = torch.tensor(datasets_test[0][epoch],
                       dtype=torch.float,
                       requires_grad=False)
y_true = torch.tensor(datasets_test[1][epoch],
                      dtype=torch.int,
                      requires_grad=False)
#y_pre1 = model(x_input)
y_pre = torch.argmax(model(x_input))
#print(y_true, y_pre1, y_pre2)
image = datasets_test[0][epoch].reshape(28, 28)
plt.imshow(image)
plt.title('label:{},predict:{}'.format(int(y_true), y_pre))
plt.show()